## 8) Dimensionality reduction and multiple regression

- EEG features showing a significant correlation to a cognitive variable were analyzed using PCA
- The latent variables obtained using PCA were used to predict the cognitive variables
- We first, used a regressor with one PC and then, we added one by one the rest PCs

Gordillo, da Cruz, Moreno, Garobbio, Herzog

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import VarianceThreshold
import statsmodels.api as sm

In [ ]:
main_dir = os.getcwd()
np.random.seed(234)

In [ ]:
# define results directories
data_dir = os.path.join(main_dir, 'data')
results_dir = os.path.join(main_dir, 'results')
results_1_dir = os.path.join(results_dir, '1_correlations_eeg_beh_results')
results_4_dir = os.path.join(results_dir, '4_correlations_groups_results')
results_8_dir = os.path.join(results_dir, '8_pca_results')

# behavior variables
beh_vars = ["Cvlt_attention_span", "Cvlt_delayed_memory", "Pts-2_subtest_3",
            "Rwt_animal_categories", "Rwt_s_words", "Tap_alertness",
            "Tap_simon_congruent", "Tap_simon_incongruent", "Tap_working_memory",
            "Tmt-A", "Tmt-B", "Vocabulary_test"]

# which group
idgroup = 'y'

In [ ]:
# Read data for PCA analysis

# results files
files_1 = os.listdir(results_1_dir)

# correlation data
sp_data = 'spearman_' + idgroup
dc_data = 'distcorr_' + idgroup
data_mask_sp = pd.read_csv(os.path.join(results_1_dir, '1_mask_' + sp_data + '.csv'), index_col=0)
data_mask_dc = pd.read_csv(os.path.join(results_1_dir, '1_mask_' + dc_data + '.csv'), index_col=0)
spearman_max = pd.read_csv(os.path.join(results_1_dir, '1_maxcorrvals_' + sp_data + '.csv'), index_col=0)
distcorr_max = pd.read_csv(os.path.join(results_1_dir, '1_maxcorrvals_' + dc_data + '.csv'), index_col=0)

In [ ]:
task = beh_vars[0]
vec_mask_sp = data_mask_sp[task].loc[data_mask_sp[task] != 'NS']
n_feats_sp = len(vec_mask_sp)
n_feats_sp
list(filter(lambda x: '1_variables_eeg_' + task + '_' + sp_data in x, files_1))

In [ ]:
# Run for each task
for k in range(len(beh_vars)):
    
    n_feats_sp = 0
    n_feats_dc = 0

    task = beh_vars[k]
    
    # load file showing which EEG features correlated significantly with the cognitive variable
    
    # Spearman correlations
    vec_mask_sp = data_mask_sp[task].loc[data_mask_sp[task] != 'NS']
    n_feats_sp = len(vec_mask_sp)
    
    # Distance correlations
    vec_mask_dc = data_mask_dc[task].loc[data_mask_dc[task] != 'NS']
    n_feats_dc = len(vec_mask_dc)
    
    data_results = []
    
    if n_feats_sp > 1:
        
        str_corr_sp = list(filter(lambda x: '1_variables_eeg_' + task + '_' + sp_data in x, files_1))
        data_results = pd.read_csv(os.path.join(results_1_dir, str_corr_sp[0]), index_col=0)
        task_out = data_results[task]
        # Power transform task values
        task_out = PowerTransformer().fit_transform(task_out.values.reshape(-1,1))
        data_results = data_results.drop([task], axis=1)
        data_index = [data_results.columns[ii][:data_results.columns[ii].find('_')] for ii in range(len(data_results.columns))]
        # Scale data
        data_ = PowerTransformer().fit_transform(data_results.values)
        pca_ = PCA(n_components=min(data_.shape))
        # Fit PCA
        pca_.fit(data_)
        
        # explained variance
        exp_variance = pd.DataFrame(pca_.explained_variance_ratio_, index=1 + np.arange(min(data_.shape)), columns=['explained variance'])
        # create df
        save_pca = pd.DataFrame(data=pca_.components_, index=1 + np.arange(min(data_.shape)), columns=data_index)
        save_pca = pd.concat([save_pca, exp_variance], axis=1)
        save_pca.to_csv(os.path.join(results_8_dir, '8_' + task + '_' + idgroup + '_pca_results_sp.csv'))
        
        # Run principal component regression
        # Transform PCA data to obtain individual scores
        X_model = pca_.transform(data_)
        X_model = pd.DataFrame(data=X_model, index=data_results.index, columns=np.arange(min(data_.shape))+1)
        
        # put each component in PCR model
        adj_r2_models = []
        comps_reg = X_model.columns
        
        # run model each time adding a new component and calculate adjusted R2 using statsmodels
        for in_pcr in range(len(comps_reg)):

            X_reg = sm.add_constant(X_model[comps_reg[0:in_pcr+1]], prepend='False')
            mod = sm.OLS(task_out, X_reg)
            res = mod.fit()
            adj_r2_models.append(res.rsquared_adj)
            
        # save PCR results
        index_df = ['PC 1-' + str(i+1) for i in range(len(adj_r2_models))]
        index_df[0] = 'PC 1'
        pcr_results = pd.DataFrame(data=adj_r2_models, columns=['adjusted R-squared'], index=index_df)
        pcr_results.to_csv(os.path.join(results_8_dir, '8_PCR_' + task + '_' + idgroup + '_sp.csv'))
    
    data_results = []
    
    if n_feats_dc > 1:
        
        str_corr_dc = list(filter(lambda x: '1_variables_eeg_' + task + '_' + dc_data in x, files_1))
        data_results = pd.read_csv(os.path.join(results_1_dir, str_corr_dc[0]), index_col=0)
        task_out = data_results[task]
        task_out = PowerTransformer().fit_transform(task_out.values.reshape(-1,1))
        data_results = data_results.drop([task], axis=1)
        data_index = [data_results.columns[ii][:data_results.columns[ii].find('_')] for ii in range(len(data_results.columns))]
        data_ = PowerTransformer().fit_transform(data_results.values)
        pca_ = PCA(n_components=min(data_.shape))
        pca_.fit(data_)
        # explained variance
        exp_variance = pd.DataFrame(pca_.explained_variance_ratio_, index=1 + np.arange(min(data_.shape)), columns=['explained variance'])
        # create df
        save_pca = pd.DataFrame(data=pca_.components_, index=1 + np.arange(len(data_index)), columns=data_index)
        save_pca = pd.concat([save_pca, exp_variance], axis=1)
        save_pca.to_csv(os.path.join(results_8_dir, '8_' + task + '_' + idgroup + '_pca_results_dc.csv'))
        
        # Run principal component regression
        # Transform data to obtain individual scores
        X_model = pca_.transform(data_)
        X_model = pd.DataFrame(data=X_model, index=data_results.index, columns=np.arange(min(data_.shape))+1)
        
        # put each component in PCR model
        adj_r2_models = []
        comps_reg = X_model.columns
        
        # run model each time adding a new component and calculate adjusted R2 using statsmodels
        for in_pcr in range(len(comps_reg)):

            X_reg = sm.add_constant(X_model[comps_reg[0:in_pcr+1]], prepend='False')
            mod = sm.OLS(task_out, X_reg)
            res = mod.fit()
            adj_r2_models.append(res.rsquared_adj)
            
        # save PCR results
        index_df = ['PC 1-' + str(i+1) for i in range(len(adj_r2_models))]
        index_df[0] = 'PC 1'
        pcr_results = pd.DataFrame(data=adj_r2_models, columns=['adjusted R-squared'], index=index_df)
        pcr_results.to_csv(os.path.join(results_8_dir, '8_PCR_' + task + '_' + idgroup + '_dc.csv'))
        

In [ ]:
# For features showing group differences

if idgroup == 'y':
    data_group = pd.read_csv(os.path.join(results_4_dir, '4_variables_eeg_y.csv'), index_col=0)
else:
    data_group = pd.read_csv(os.path.join(results_4_dir, '4_variables_eeg_o.csv'), index_col=0)
        

# PCA
data_ = VarianceThreshold().fit_transform(data_group.values)
data_ = PowerTransformer().fit_transform(data_)
pca_group = PCA(n_components=min(data_group.shape))
pca_group.fit(data_)
# explained variance
exp_variance = pd.DataFrame(pca_group.explained_variance_ratio_, index=1 + np.arange(min(data_group.shape)), columns=['explained variance'])
# concatenate DataFrame
save_pca = pd.DataFrame(data=pca_group.components_, index=1 + np.arange(min(data_group.shape)), columns=data_group.columns)
save_pca = pd.concat([save_pca, exp_variance], axis=1)
save_pca.to_csv(os.path.join(results_8_dir, '8_group_difference' + '_' + idgroup + '_pca_results_sp.csv'))
